# 청와대 청원 크롤링 코드 

## 작성자: 박하람
## 작성일자: 2020년 8월 10일 

분야별 청원 전체 카테고리 선택 후, 추천 순 정렬하여 모든 청원 가져오기

In [47]:
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")

In [245]:
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time

In [393]:
driver = webdriver.Chrome('/Users/harampark/Documents/chromedriver')
driver.get('https://www1.president.go.kr/petitions')

In [394]:
# 만료된 청원 클릭 
expired_petition = driver.find_element_by_class_name('category_tab2')
expired_petition.send_keys(Keys.ENTER)

In [395]:
# 추천순보기 클릭 
latest_views = driver.find_element_by_id("order")
latest_views.send_keys('추천순 보기')

In [396]:
def get_petition_list(): 
    
    category = []
    titles = []
    address = []
    end_date = []
    consent_num = []
    
    res = driver.page_source 
    soup = BeautifulSoup(res, "html.parser")
    
    contents = soup.find_all('ul', class_='petition_list')
    
    for i in range(7):
        list = contents[0].find_all('div', class_='bl_wrap')[i]
    
        # 청원 카테고리
        category.append(list.find('div', class_='bl_category').get_text().split()[1]) 
        # 청원 제목 
        titles.append(list.find('div', class_='bl_subject').get_text().split('제목 ')[1])
        # 청원 주소 
        add_url = list.find('a')['href']
        address.append('https://www1.president.go.kr'+add_url)
        # 청원 만료일 
        end_date.append(list.find('div', class_='bl_date').get_text().split()[2])
        # 청원 동의 수 
        consent_num.append(list.find('div', class_='bl_agree').get_text().split()[1].split('명')[0])
    
    return category, titles, address, end_date, consent_num 
    

In [397]:
def get_petitions(start, end): 
    
    category = []
    titles = []
    address = []
    end_date = []
    consent_num = []
    
    for page_num in range(start, int(end/10)+1): 
        for n in range(1,11): 
            # 1 page 가져와서 넣어주기 
            page_info = get_petition_list()
            category.append(page_info[0])
            titles.append(page_info[1])
            address.append(page_info[2])
            end_date.append(page_info[3])
            consent_num.append(page_info[4])
            print(n,'페이지 완료')

            # 페이지 바꿔주기 
            if n != 10:    
                p_btn = driver.find_elements_by_class_name('p_btn')
                if start != 1: 
                    p_btn_num = p_btn[1].find_elements_by_tag_name('a')
                    time.sleep(1)
                else:
                    p_btn_num = p_btn[0].find_elements_by_tag_name('a')
                    time.sleep(2)

                # 페이지 버튼 클릭 
                if n%2 != 0:
                    p_btn_num[n].click()
                    time.sleep(1)
                else:
                    p_btn_num[n].click()
                    time.sleep(2)
                time.sleep(0.5)
            else: 
                p_btn_next = driver.find_elements_by_class_name('p_btn.next')
                p_btn_next[0].click()
                print(start*10,'pages 완료 -> Next')
       
        
    return category, titles, address, end_date, consent_num
 
   

In [398]:
def get_petition_df(start, end): 
    petitions_raw = get_petitions(start, end)
    
    for n in range(len(list[0])): 
        cate += list[0][n]
        tit += list[1][n]
        add += list[2][n]
        end += list[3][n]
        con += list[4][n]
    
    petition_dict = {'카테고리':cate, '제목':tit, '청원종료일': end,
       '참여인원': con,'URL': add}
    df = pd.DataFrame(petition_dict)
    
    return df

In [399]:
petition_df = get_petition_df(1,20)
petition_df

1 페이지 완료
2 페이지 완료
3 페이지 완료
4 페이지 완료
5 페이지 완료
6 페이지 완료
7 페이지 완료
8 페이지 완료
9 페이지 완료
10 페이지 완료
10 pages 완료 -> Next


IndexError: list index out of range

In [392]:
driver.close()